# Model

In [ ]:
import numpy as np

In [ ]:
class Layer_Input:
    def forward(self,inputs):
        self.output = inputs

In [ ]:
class Activation_Softmax:
    def forward(self,inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
        self.output = exp_values / np.sum(exp_values,axis=1,keepdims=True)
        
    def backward(self,dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output,single_dvalues) in enumerate(zip(self.output,dvalues)):
            single_output = single_output.reshape(-1,1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output,single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)
    
    def predictions(self,outputs):
        return np.argmax(outputs,axis=1)

In [ ]:
class Activation_Sigmoid:
    def forward(self,inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-self.inputs))
    
    def backward(self,dvalues):
        self.dinputs = dvalues * (1-self.output) * self.output
        
    def predictions(self,outputs):
        return (outputs > 0.5) * 1

In [ ]:
class Activation_Linear:
    def forward(self,inputs):
        self.inputs = inputs
        self.output = inputs
        
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        
    def predictions(self,outputs):
        return outputs

In [ ]:
class Activation_RELU:
    def forward(self,inputs):
        self.inputs = inputs
        self.output = np.maximum(0,inputs)
        
    def backward(self,dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0
    
    def predictions(self,outputs):
        return outputs

In [ ]:
class Loss:
    def regularization_loss(self,layer):
        regularization_loss = 0
        # Weight Loss
        for layer in self.trainable_layers:
            if layer.weight_regularizer_l1 > 0:
                # sum of absolute values of weights
                regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
            if layer.weight_regularizer_l2 > 0:
                # sum of squared values of weights
                regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)
            if layer.bias_regularizer_l1 > 0:
                regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
            if layer.bias_regularizer_l2 > 0:
                regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)
            
        return regularization_loss
    
    
    def remember_trainable_layers(self,layers):
        self.trainable_layers = layers
        
    def calculate(self,output,y,*,include_regularization=False):
        sample_losses = self.forward(output,y)
        data_loss = np.mean(sample_losses)
        if not include_regularization:
            return data_loss
        return data_loss,self.regularization_loss()

In [ ]:
class Accuracy:
    def calculate(self,predictions,y):
        comparisions = self.compare(predictions,y)
        accuracy = np.mean(comparisions)
        return accuracy

In [ ]:
class Accuracy_Regression:
    def __init__(self):
        self.predictions = None
        
    def init(self,y,reinit=False):
        if self.predictions is None or reinit:
            self.predictions = np.std(y) / 250
    
    def compare(self,predictions,y):
        return np.absolute(predictions - y) < self.predictions

In [ ]:
class Accuracy_Categorical:
    def init(self,y):
        pass
    
    def compare(self,predicitons,y):
        if len(y.shape) == 2:
            y = np.argmax(y,axis=1)
        return predicitons == y

In [ ]:
class Model:
    def __init__(self):
        self.layers = []
        
    def add(self,layer):
        self.layers.append(layer)
    
    def set(self,*,loss,optimizer,accuracy):
        self.loss = loss
        self.optimizer = optimizer
        self.accuarcy = accuracy
        
    def finalize(self):
        self.input_layer = Layer_Input()
        layer_count = len(self.layers)
        self.trainable_layers = []
        
        for i in range(layer_count):
            if i==0:
                self.layers[i].prev = self.input_layer
                self.layers[i].next = self.layers[i+1]
            elif i < layer_count - 1:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.layers[i+1]
            else:
                self.layers[i].prev = self.layers[i-1]
                self.layers[i].next = self.loss
                self.output_layer_activation = self.layers[i]
            
            if hasattr(self.layers[i],'weights'):
                self.trainable_layers.append(self.layers[i])
        
        self.loss.remember_trainable_layers(self.trainable_layers)
    
    def train(self,X,y,*, epochs=1,print_every=1):
        self.accuarcy.init(y)
        
        for epoch in range(1,epochs+1):
            output = self.forward(X)
            data_loss,reg_loss = self.loss.calculate(output,y,include_regularization=True) 
            loss = data_loss + reg_loss
            predictions = self.output_layer_activation.predictions(output)
            accuracy = self.accuarcy.calculate(predictions,y)
            self.backward(output,y)
            
            self.optimizer.pre_update_params()
            for layer in self.trainable_layers:
                self.optimizer.update_params(layer)
            self.optimizer.post_update_params()
            
            if not epoch % print_every:
                print(f"Epoch : {epoch} acc: {accuracy} loss: {loss}")
        
    def forward(self,X):
        self.input_layer.forward(X)
        for layer in self.layers:
            layer.forward(layer.prev.output)
            
        return layer.output
    
    def backward(self,output,y):
        self.loss.backward(output,y)
        for layer in reversed(self.layers):
            layer.backward(layer.next.dinputs)